In [ ]:
!pip install sqlalchemy
!pip install mysql-connector


In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime

#Load into dataframe
df = pd.read_csv('./sample_data/Amazon Sale Report.csv', encoding='utf-8', index_col="index")

# See the Schema, structure, data types of the dataset
df.info()

#Check Missing Values
print(df.isnull().sum())


<ipython-input-1-66746b9394bf>:7: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./sample_data/Amazon Sale Report.csv', encoding='utf-8', index_col="index")


<class 'pandas.core.frame.DataFrame'>
Index: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Order ID            128975 non-null  object 
 1   Date                128975 non-null  object 
 2   Status              128975 non-null  object 
 3   Fulfilment          128975 non-null  object 
 4   Sales Channel       128975 non-null  object 
 5   ship-service-level  128975 non-null  object 
 6   Style               128975 non-null  object 
 7   SKU                 128975 non-null  object 
 8   Category            128975 non-null  object 
 9   Size                128975 non-null  object 
 10  ASIN                128975 non-null  object 
 11  Courier Status      122103 non-null  object 
 12  Qty                 128975 non-null  int64  
 13  currency            121180 non-null  object 
 14  Amount              121180 non-null  float64
 15  ship-city           128942 non-null  ob

In [2]:
#drop unnecessary columns
df.drop(columns=['Unnamed: 22', 'fulfilled-by', 'ship-country', 'currency'], inplace = True)

#Replace space and - with underscore for column names to run sql query successfully
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')

# Check missing values and replace
print(df.isnull().sum())

Order_ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales_Channel_            0
ship_service_level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier_Status         6872
Qty                       0
Amount                 7795
ship_city                33
ship_state               33
ship_postal_code         33
promotion_ids         49153
B2B                       0
dtype: int64


In [4]:
################## cleaning and Tranformation #################################################
#drop unnecessary columns
#df.drop(columns=['Unnamed: 22', 'fulfilled-by', 'ship-country', 'currency'], inplace = True)

#Replace space and - with underscore for column names to run sql query successfully
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')

# Check missing values and replace
print(df.isnull().sum())

#replace Courier Status = Cancelled and Amount = 0 where null
df['Courier_Status'] = df.apply(lambda row: 'Cancelled' if row['Qty'] == 0 else row['Courier_Status'], axis=1)
df['Amount'] = df.apply(lambda row: 0 if row['Qty'] == 0 else row['Amount'], axis=1)

#Replace with Nan for following columns
cols = ['ship_city', 'ship_state', 'ship_postal_code', 'promotion_ids']
df[cols] = df[cols].map(lambda row: "NaN" if pd.isnull(row) else row)

# Verify all missing values is replaced now
print(df.isnull().sum())

#convert date to same format for all rows
df['Date'] = pd.to_datetime(df['Date'], format='mixed').dt.strftime('%Y/%m/%d')
#Extract month in sepearte column
df['month'] = pd.to_datetime(df['Date']).dt.month

df.to_csv('output_file.csv', index=True)


Order_ID                0
Date                    0
Status                  0
Fulfilment              0
Sales_Channel_          0
ship_service_level      0
Style                   0
SKU                     0
Category                0
Size                    0
ASIN                    0
Courier_Status          0
Qty                     0
Amount                124
ship_city               0
ship_state              0
ship_postal_code        0
promotion_ids           0
B2B                     0
month                   0
dtype: int64
Order_ID                0
Date                    0
Status                  0
Fulfilment              0
Sales_Channel_          0
ship_service_level      0
Style                   0
SKU                     0
Category                0
Size                    0
ASIN                    0
Courier_Status          0
Qty                     0
Amount                124
ship_city               0
ship_state              0
ship_postal_code        0
promotion_ids           0

In [5]:
from sqlalchemy import create_engine
#Create a database
engine = create_engine('sqlite:///amazonsales.db', echo=False)
df.to_sql('sales_data', con=engine, if_exists='replace', index=False)

#1 Category Performance:
# Which product categories generate the highest revenue and quantity sold?
# Are there any low-performing categories that need attention?
querie1 = 'SELECT Category, sum(Qty) as Quantity, SUM(Qty * Amount) AS total_revenue FROM sales_data GROUP BY Category order by total_revenue desc'
result = pd.read_sql(querie1, engine)
print(result)

        Category  Quantity  total_revenue
0            Set     45289     37934434.0
1          kurta     45045     20675349.0
2  Western Dress     13943     10707932.0
3            Top      9903      5242931.0
4   Ethnic Dress      1053       762949.0
5         Blouse       863       441259.0
6         Bottom       398       142870.0
7          Saree       152       125767.0
8        Dupatta         3          915.0


In [6]:
#2.Seasonal Sales Trends: What are the monthly and weekly sales trends?
# monthly sales
# 4th month highest sales
querie2 = 'SELECT month, SUM(Qty * Amount) AS total_sales FROM sales_data WHERE Courier_Status = "Shipped" GROUP BY month ORDER BY total_sales desc'
result = pd.read_sql(querie2, engine)
#print(result)
# Set is most selling product in 4th month
querie3 = 'SELECT month, count(month), Category, SUM(Qty * Amount) AS monthly_total_sales FROM sales_data GROUP BY Category ORDER BY monthly_total_sales desc'
result = pd.read_sql(querie3, engine)
print(result)

   month  count(month)       Category  monthly_total_sales
0      4         50284            Set           37934434.0
1      4         49877          kurta           20675349.0
2      4         15500  Western Dress           10707932.0
3      4         10622            Top            5242931.0
4      4          1159   Ethnic Dress             762949.0
5      4           926         Blouse             441259.0
6      4           440         Bottom             142870.0
7      4           164          Saree             125767.0
8      6             3        Dupatta                915.0


In [7]:
#3 Fulfillment Efficiency:
# how do sales and revenue differ across fulfillment methods (Amazon Fulfilled vs. Seller Fulfilled)?
querie4 = 'SELECT Fulfilment, Count(Fulfilment), SUM(Amount) As total_sales, SUM(Qty * Amount) as Revenue, Courier_Status FROM sales_data GROUP BY Fulfilment, Courier_Status ORDER BY total_sales desc'
result = pd.read_sql(querie4, engine)
print(result)
#4. Product Size Analysis:
# which sizes are most popular in terms of quantity sold?
querie5 = 'SELECT Size, count(Qty) FROM sales_data GROUP BY Size ORDER BY count(Qty) desc'
result = pd.read_sql(querie5, engine)
print(result)


  Fulfilment  Count(Fulfilment)  total_sales     Revenue Courier_Status
0     Amazon              77606   50331934.0  50692128.0        Shipped
1   Merchant              31881   20726730.0  20961166.0        Shipped
2     Amazon               6157    3990217.0   4022019.0      Unshipped
3   Merchant                524     353625.0    359093.0      Unshipped
4     Amazon               5935          0.0         0.0      Cancelled
5   Merchant               6872          0.0         0.0      Cancelled
    Size  count(Qty)
0      M       22711
1      L       22132
2     XL       20876
3    XXL       18096
4      S       17090
5    3XL       14816
6     XS       11161
7    6XL         738
8    5XL         550
9    4XL         427
10  Free         378


In [8]:
# 5 Courier Impact on Revenue:
#What is the percentage of lost or delayed shipments? (we dont have lost, delayed in the data set so used "Shipped","Cancelled")
querie6 ='SELECT Courier_Status, (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM sales_data)) AS percentage FROM sales_data WHERE Courier_Status IN ("Shipped","Cancelled") GROUP BY Courier_Status'
result = pd.read_sql(querie6, engine)
print(result)
#6 Are there any specific categories or styles that perform better in B2B sales?
querie7 = 'select Category, count(Category) As total_products, Max(Qty), Courier_Status, B2B from sales_data where B2B = 1 group by Category, B2B order by Category'
result = pd.read_sql(querie7, engine)
print(result)

  Courier_Status  percentage
0      Cancelled    9.929831
1        Shipped   84.890095
        Category  total_products  Max(Qty) Courier_Status  B2B
0         Blouse               8         1        Shipped    1
1         Bottom               4         1        Shipped    1
2   Ethnic Dress              15         1        Shipped    1
3          Saree               2         1        Shipped    1
4            Set             340         5        Shipped    1
5            Top              61         1        Shipped    1
6  Western Dress             113         2        Shipped    1
7          kurta             328         3        Shipped    1


In [10]:
#Top 5 products by revenue.
querie8 = '''select * from (SELECT SKU, Category, SUM(Qty * Amount) AS total_revenue, RANK()
OVER (order BY SUM(Qty * Amount) DESC) As rank FROM sales_data WHERE Courier_Status = "Shipped" GROUP BY Category order by rank) data
where rank <= 5'''
result = pd.read_sql(querie8, engine)
print(result)

#Top 5 products by quantity sold.
querie9 = '''select * from (SELECT SKU, Category, SUM(Qty) AS total_quantity, RANK()
OVER (order BY SUM(Qty) DESC) As rank FROM sales_data GROUP BY Category order by rank) data where rank <= 5'''
result = pd.read_sql(querie9, engine)
print(result)

               SKU       Category  total_revenue  rank
0  SET264-KR-NP-XL            Set     35696431.0     1
1  JNE3781-KR-XXXL          kurta     19486464.0     2
2       J0341-DR-S  Western Dress     10159456.0     3
3  JNE3671-TU-XXXL            Top      4899409.0     4
4     J0211-DR-XXL   Ethnic Dress       729278.0     5
               SKU       Category  total_quantity  rank
0   SET389-KR-NP-S            Set           45289     1
1  JNE3781-KR-XXXL          kurta           45045     2
2       J0341-DR-L  Western Dress           13943     3
3  JNE3671-TU-XXXL            Top            9903     4
4     J0211-DR-XXL   Ethnic Dress            1053     5
